In [1]:
from __future__ import unicode_literals, print_function, division
import torch
import torch.nn as nn
from io import open
import glob
import os
import random


In [2]:
def findFiles(path): return glob.glob(path)

print(findFiles('data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

['data/names/Czech.txt', 'data/names/German.txt', 'data/names/Arabic.txt', 'data/names/Japanese.txt', 'data/names/Chinese.txt', 'data/names/Vietnamese.txt', 'data/names/Russian.txt', 'data/names/French.txt', 'data/names/Irish.txt', 'data/names/English.txt', 'data/names/Spanish.txt', 'data/names/Greek.txt', 'data/names/Italian.txt', 'data/names/Portuguese.txt', 'data/names/Scottish.txt', 'data/names/Dutch.txt', 'data/names/Korean.txt', 'data/names/Polish.txt']
Slusarski


In [3]:
print(category_lines['Italian'][:5])

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


In [4]:
# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for idx, letter in enumerate(line):
        tensor[idx][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Jones').size())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])
torch.Size([5, 1, 57])


In [5]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [6]:
input = letterToTensor('A')
print('input', input.shape)
hidden = torch.zeros(1, n_hidden)
print('init_hidden', hidden.shape)

output, next_hidden = rnn(input, hidden)

print('output', input.shape)
print('output hidden', next_hidden.shape)


input torch.Size([1, 57])
init_hidden torch.Size([1, 128])
output torch.Size([1, 57])
output hidden torch.Size([1, 128])


In [7]:
input = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)
print('input', input.shape)

output, next_hidden = rnn(input[0], hidden)
print('output', output.shape)
print('output hidden', next_hidden.shape)

input torch.Size([6, 1, 57])
output torch.Size([1, 18])
output hidden torch.Size([1, 128])


In [9]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

('Portuguese', 13)


In [11]:
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = Portuguese / line = Simoes
category = Arabic / line = Aswad
category = Czech / line = Cernohous
category = Czech / line = Hawlata
category = Korean / line = Chweh
category = German / line = Kruger
category = Arabic / line = Bishara
category = Italian / line = Goretti
category = Scottish / line = Kennedy
category = German / line = Hirsch


In [12]:
category, line, category_tensor, line_tensor = randomTrainingExample()
print('category =', category, '/ line =', line)

category = Irish / line = Nevin


In [15]:
criterion = nn.NLLLoss()

In [16]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()

In [17]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000

# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0


5000 5% (0m 2s) 2.1224 Vuu / Vietnamese ✓
10000 10% (0m 5s) 3.5081 Hay / Chinese ✗ (Scottish)
15000 15% (0m 7s) 1.0754 Saromi / Japanese ✓
20000 20% (0m 10s) 1.0341 Thao / Chinese ✗ (Vietnamese)
25000 25% (0m 13s) 0.6019 Novoselov / Russian ✓
30000 30% (0m 15s) 2.2343 Forestier / German ✗ (French)
35000 35% (0m 18s) 3.0358 Rhys / Korean ✗ (Irish)
40000 40% (0m 20s) 2.1944 Rocha / Spanish ✗ (Portuguese)
45000 45% (0m 23s) 0.1227 Musahanov / Russian ✓
50000 50% (0m 26s) 0.3059 Seok / Korean ✓
55000 55% (0m 28s) 0.6020 Suk / Korean ✓
60000 60% (0m 31s) 0.3399 Lam / Vietnamese ✓
65000 65% (0m 33s) 1.4336 Fournier / French ✓
70000 70% (0m 36s) 2.0691 Betlach / German ✗ (Czech)
75000 75% (0m 39s) 2.7436 Mclain / Scottish ✗ (Irish)
80000 80% (0m 41s) 0.2217 Aswad / Arabic ✓
85000 85% (0m 44s) 2.2695 Salazar / Arabic ✗ (Portuguese)
90000 90% (0m 47s) 0.2368 Privalikhin / Russian ✓
95000 95% (0m 50s) 0.9930 Totah / Arabic ✓
100000 100% (0m 52s) 0.4506 Rudaski / Polish ✓
